In [11]:
#Convert K-Map derived Sum of Product Equations into Dataflow verilog Code.

In [63]:
import numpy as np
import time

In [152]:
LED_equations = np.array(["D[0] = Y₃Y₂' + Y₃Y₁' + Y₃Y₀' + Y₃'Y₂Y₁Y₀",
        "D[1] = Y₂Y₁' + Y₂Y₀' + Y₂'Y₁Y₀", "D[2] = Y₁'Y₀ + Y₁Y₀'",
        "D[3] = Y₀'"])
HEX_equations = np.array([["D[0] = Y₃'Y₁' + Y₂'Y₀ + Y₁'Y₀ + Y₃Y₁",
        "D[1] = Y₁'Y₀' + Y₂Y₁' + Y₂Y₀' + Y₃Y₁ + Y₃'Y₂'Y₀",
        "D[2] = Y₃Y₀' + Y₃Y₂ + Y₃'Y₂'Y₀ + Y₃'Y₁'Y₀",
        "D[3] = Y₀ + Y₁ + Y₃'Y₂", "D[4] = Y₂Y₁'Y₀ + Y₃Y₂Y₁",
        "D[5] = Y₃'Y₁'Y₀ + Y₃'Y₂Y₁' + Y₂Y₁'Y₀ + Y₂Y₁Y₀' + Y₃Y₁Y₀",
        "D[6] = Y₃Y₂Y₁ + Y₃Y₂'Y₀' + Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀"],
       ["D[0] = Y₃'Y₁' + Y₃'Y₀' + Y₂Y₀' + Y₃Y₁",
        "D[1] = Y₂ + Y₃'Y₁' + Y₃'Y₀' + Y₃Y₀",
        "D[2] = Y₃'Y₀' + Y₃Y₀ + Y₂Y₁Y₀'", "D[3] = Y₀' + Y₁ + Y₃'Y₂",
        "D[4] = Y₂Y₁Y₀' + Y₃Y₂Y₁ + Y₃'Y₂'Y₁'Y₀'",
        "D[5] = Y₂Y₁ + Y₃'Y₂'Y₀' + Y₃'Y₂Y₀ + Y₃Y₂Y₀'",
        "D[6] = Y₃'Y₁'Y₀' + Y₂Y₁Y₀' + Y₃Y₂'Y₀ + Y₃Y₁Y₀"],
       ["D[0] = Y₃'Y₂' + Y₃'Y₀ + Y₃'Y₁ + Y₁Y₀ + Y₃Y₂Y₁'",
        "D[1] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'",
        "D[2] = Y₃'Y₀ + Y₃'Y₂'Y₁' + Y₂Y₁Y₀ + Y₃Y₁Y₀' + Y₃Y₂Y₀'",
        "D[3] = Y₁' + Y₀ + Y₃'Y₂", "D[4] = Y₃'Y₂'Y₁' + Y₂Y₁Y₀",
        "D[5] = Y₂'Y₁'Y₀' + Y₃'Y₂'Y₀ + Y₃'Y₂Y₁ + Y₃Y₂Y₀",
        "D[6] = Y₃'Y₂'Y₁' + Y₃'Y₁'Y₀ + Y₂Y₁Y₀ + Y₃Y₂'Y₁Y₀' + Y₃Y₂Y₁'Y₀'"],
       ["D[0] = Y₃'Y₂' + Y₁'Y₀' + Y₃'Y₁ + Y₂Y₀' + Y₃Y₂Y₁'",
        "D[1] = Y₃'Y₂' + Y₂'Y₁' + Y₃'Y₀' + Y₂Y₁ + Y₃Y₀",
        "D[2] = Y₃'Y₀' + Y₃'Y₂'Y₁' + Y₂'Y₁'Y₀' + Y₃Y₁Y₀ + Y₃Y₂Y₀",
        "D[3] = Y₁' + Y₀' + Y₃'Y₂",
        "D[4] = Y₃'Y₂'Y₁' + Y₃'Y₂'Y₀' + Y₂'Y₁'Y₀'",
        "D[5] = Y₂'Y₁' + Y₃'Y₂'Y₀' + Y₃'Y₁'Y₀' + Y₃'Y₂Y₁Y₀ + Y₃Y₂Y₁Y₀'",
        "D[6] = Y₃'Y₂'Y₁' + Y₂'Y₁'Y₀' + Y₃'Y₁Y₀' + Y₃Y₂'Y₁Y₀ + Y₃Y₂Y₁'Y₀"]])

In [153]:
def parse_line(equation, inpt, output):
    """
    Parses individual equation to verilog equation.
    Parameter:
    1. equation: The equation to convert into verilog code.
    2. inpt: The input variable names.
    3. output: The output variable names.
    Returns:
    1. verilog: The corresponding verilog code
    """
    index = 0
    counter = 0
    
    result = equation.split(" = ")[0]
    
    f_x = ""
    expression = ""
    
    for res in result:
        if(res.isnumeric()):
            f_x += str(int(res))
        else:
            if(res == 'D'):
                f_x += inpt
            else:
                f_x += res
                
    express = equation.split(" =")[1] + " "
    SUB_NORM = str.maketrans("₀₁₂₃₄₅₆₇₈₉",  "0123456789")
    express = express.translate(SUB_NORM)
    express = express.replace("+", "|")
    express = express.replace("Y", output)
    
    for i in range(len(express)-1):
        if(express[i+1] == "'"):
            expression += "(" + "~" + output + "[" + str(int(express[i])) + "])" 
        elif(express[i+1] != "'" and express[i].isnumeric()):
            expression += output + "[" + str(int(express[i])) + "]"
        if(express[i] == "|"):
            expression += ") | ("
        
    value = ""
    for j in range(len(expression)-1):
        value += expression[j]
        if(expression[j] == "]" and expression[j+1] == output[0]):
            value += " & "
        if(expression[j] == ")" and expression[j+1] == output[0]):
            value += " & "
        if(expression[j] == "]" and expression[j+1] == "("):
            value += " & "
        if(expression[j] == ")" and expression[j+1] == "("):
            value += " & "
    value = value + expression[-1]
    return "assign " + f_x + " = " + "(" + value + ");"

In [154]:
def equation_convert(set_equations, inpt, out):
    """
    Converts set of boolean expressions into dataflow verilog.
    """
    equations = []
    for one in set_equations:
        eq = parse_line(one, inpt, out)
        equations.append(eq)
    return np.array(equations)

In [155]:
def HEX_equation_convert(hex_set, inpt, output):
    """
    Converts series of HEX equations to dataflow verilog structure.
    Returns verilog code.
    """
    hex_code = []
    for set_0 in hex_set:
        a_set = equation_convert(set_0, inpt, output)
        hex_code.append(a_set)
    return np.array(hex_code)

In [156]:
LED_verilog = equation_convert(LED_equations, "next", "present")

In [157]:
HEX_verilog = HEX_equation_convert(HEX_equations, "out", "in")

In [158]:
def to_nands(verilog, mult=False):
    """Converts code to NANDS and Inverters"""
    NAND = []
    if(mult == True):
        for xx in verilog:
            ind_nand = []
            for cc in xx:
                one_eq = cc.replace("| ", "& ~")
                one_eq = one_eq.replace("= ", "= ~( ~")
                one_eq = one_eq.replace(";", " );")
                ind_nand.append(one_eq)
            NAND.append(ind_nand)
    else:
        for cc in verilog:
            one_eq = cc.replace("| ", "& ~")
            one_eq = one_eq.replace("= ", "= ~( ~")
            one_eq = one_eq.replace(";", " );")
            NAND.append(one_eq)
    return np.array(NAND)

In [159]:
LED_verilog_NANDS = to_nands(LED_verilog)

In [160]:
print("LED Equations\n\n\n")
for i in range(len(LED_verilog_NANDS)):
    print(LED_verilog_NANDS[i], "\n")

LED Equations



assign next[0] = ~( ~(present[3] & (~present[2])) & ~(present[3] & (~present[1])) & ~(present[3] & (~present[0])) & ~((~present[3]) & present[2] & present[1] & present[0]) ); 

assign next[1] = ~( ~(present[2] & (~present[1])) & ~(present[2] & (~present[0])) & ~((~present[2]) & present[1] & present[0]) ); 

assign next[2] = ~( ~((~present[1]) & present[0]) & ~(present[1] & (~present[0])) ); 

assign next[3] = ~( ~((~present[0])) ); 



In [161]:
LED_verilog

array(['assign next[0] = (present[3] & (~present[2])) | (present[3] & (~present[1])) | (present[3] & (~present[0])) | ((~present[3]) & present[2] & present[1] & present[0]);',
       'assign next[1] = (present[2] & (~present[1])) | (present[2] & (~present[0])) | ((~present[2]) & present[1] & present[0]);',
       'assign next[2] = ((~present[1]) & present[0]) | (present[1] & (~present[0]));',
       'assign next[3] = ((~present[0]));'], dtype='<U165')

In [162]:
HEX_verilog_NAND = to_nands(HEX_verilog, True)

In [163]:
print("HEX Equations\n\n\n")
for i in range(len(HEX_verilog_NAND)):
    print("\n---------------------------\n", "SSD" + str(i) + ".v\n\n")
    for j in range(len(HEX_verilog_NAND[i])):
        print(HEX_verilog_NAND[i][j], "\n")

HEX Equations




---------------------------
 SSD0.v


assign out[0] = ~( ~((~in[3]) & (~in[1])) & ~((~in[2]) & in[0]) & ~((~in[1]) & in[0]) & ~(in[3] & in[1]) ); 

assign out[1] = ~( ~((~in[1]) & (~in[0])) & ~(in[2] & (~in[1])) & ~(in[2] & (~in[0])) & ~(in[3] & in[1]) & ~((~in[3]) & (~in[2]) & in[0]) ); 

assign out[2] = ~( ~(in[3] & (~in[0])) & ~(in[3] & in[2]) & ~((~in[3]) & (~in[2]) & in[0]) & ~((~in[3]) & (~in[1]) & in[0]) ); 

assign out[3] = ~( ~(in[0]) & ~(in[1]) & ~((~in[3]) & in[2]) ); 

assign out[4] = ~( ~(in[2] & (~in[1]) & in[0]) & ~(in[3] & in[2] & in[1]) ); 

assign out[5] = ~( ~((~in[3]) & (~in[1]) & in[0]) & ~((~in[3]) & in[2] & (~in[1])) & ~(in[2] & (~in[1]) & in[0]) & ~(in[2] & in[1] & (~in[0])) & ~(in[3] & in[1] & in[0]) ); 

assign out[6] = ~( ~(in[3] & in[2] & in[1]) & ~(in[3] & (~in[2]) & (~in[0])) & ~(in[2] & (~in[1]) & in[0]) & ~((~in[3]) & (~in[2]) & in[1] & in[0]) ); 


---------------------------
 SSD1.v


assign out[0] = ~( ~((~in[3]) & (~in[1])) & ~((~in